In [158]:
import pandas as pd
df1 = pd.read_csv('test.csv')
df1.head()

,movieid,title,genres,overview,string_agg
0,3,天堂孤影,"剧情,喜剧",環衛公司垃圾車駕駛員尼卡德（Matti Pellonp鳵 飾）的老同事計劃籌資組建自己的環衛...,"Safka Pekkonen,Aki Kaurismäki,Jukka-Pekka Palo..."
1,11,星際大戰四部曲：曙光乍現,"科幻,冒险,动作",遙遠星係發生叛亂，銀河共和國被推翻，奧爾德蘭星的莉亞公主（卡里•費甚爾Carrie Fish...,"Alan Harris,John Chapman,Colin Michael Kitchen..."
2,12,海底總動員,"家庭,动画",小丑魚馬林（Marlin）與配偶珊瑚（Coral）於大堡礁建立了家園，但一場梭子魚的攻擊使馬...,"Alexander Gould,Joe Ranft,Elizabeth Perkins,Br..."
3,13,阿甘正傳,"爱情,剧情,喜剧",二次大戰剛結束，阿甘出生在美國阿拉巴馬州的一個閉塞小鎮，他先天弱智，但上帝又賜予他一雙疾步如...,"Steven Griffith,Isabel Rose,Byron Minns,Don Fi..."
4,14,美國心玫瑰情,剧情,故事的主人翁賴斯特罕住在一個典型的美國小鎮裡，他面臨中年危機，突然對自己的生活感到不滿：和太...,"Peter Gallagher,Amber Smith,Sam Robards,Ara Ce..."


In [159]:
def removeComma(strs):
    return strs.split(",")

In [160]:
from opencc import OpenCC

def list_genres(x):
    cc = OpenCC('s2tw')
    l = [cc.convert(x)]
    return(l)
df1['genres'] = df1['genres'].apply(list_genres)
df1['genres'].head()

0       [劇情,喜劇]
1    [科幻,冒險,動作]
2       [家庭,動畫]
3    [愛情,劇情,喜劇]
4          [劇情]
Name: genres, dtype: object

In [161]:
#df1['genres'] = df1['genres'].apply(removeComma)

In [162]:
def list_cast(listString):
   return listString.split(",")
df1['string_agg'] = df1['string_agg'].apply(list_cast)

In [163]:
def list_cast(x):
    if len(x) > 3:
        x = x[:1]
    return(x)
df1['string_agg'] = df1['string_agg'].apply(list_cast)
df1['string_agg'].head()

0     [Safka Pekkonen]
1        [Alan Harris]
2    [Alexander Gould]
3    [Steven Griffith]
4    [Peter Gallagher]
Name: string_agg, dtype: object

In [164]:
missing = df1.columns[df1.isnull().any()]
df1[missing].isnull().sum().to_frame()

# Replace NaN from overview with an empty string
df1['overview'] = df1['overview'].fillna('')

In [165]:
import jieba
import jieba.analyse
#split overview
#print(df1['overview'])
jieba.set_dictionary('./dict.txt')
overviews = []
for d in df1['overview'].astype(str):
    keywords = jieba.analyse.extract_tags(d, topK=6)
    overviews.append(keywords)

df1['keyword'] = overviews

Building prefix dict from /Users/zhangxinyu/nlp-pro/dict.txt ...
Loading model from cache /var/folders/g9/__yhhv7d0xx59_jssnmpjhwh0000gn/T/jieba.ufb30e0fe4a3317c83cabe51d90c31569.cache
Loading model cost 0.530 seconds.
Prefix dict has been built successfully.


In [166]:
import re

df1['feature'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['title'])) + ' ' + ' '.join(x['genres'])+ ' ' +  ' '.join(x['keyword'])+ ' ' )
df1['feature'] = df1.apply(bag_words, axis = 1)

#df1['feature'].head()

feature = df1[['movieid','title','genres','string_agg','feature']]
feature.to_csv('feature.csv',index=False)

In [219]:
import pandas as pd
df1 = pd.read_csv(u'feature.csv')
df1.head()

,movieid,title,genres,string_agg,feature
0,3,天堂孤影,"['劇情,喜劇']",['Safka Pekkonen'],"天堂孤影 劇情,喜劇 卡德 羅娜 環衛 經理 服裝店 同事"
1,11,星際大戰四部曲：曙光乍現,"['科幻,冒險,動作']",['Alan Harris'],"星際大戰四部曲曙光乍現 科幻,冒險,動作 銀河 盧克 帝國 機器人 奧爾德 絕地"
2,12,海底總動員,"['家庭,動畫']",['Alexander Gould'],"海底總動員 家庭,動畫 馬林 尼莫 卵子 攻擊 潛水員 珊瑚"
3,13,阿甘正傳,"['愛情,劇情,喜劇']",['Steven Griffith'],"阿甘正傳 愛情,劇情,喜劇 阿甘 美國 一個 小鎮 愛護 尼克森"
4,14,美國心玫瑰情,['劇情'],['Peter Gallagher'],美國心玫瑰情 劇情 小珍 卡洛琳 於是 越來越 人生 罕住


In [220]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
# 計算個詞語出現的次數
cv_mx = cv.fit_transform(df1['feature'])

# create cosine similarity matrix 餘弦相似度
cosine_sim = cosine_similarity(cv_mx, cv_mx)
print(cosine_sim)

[[1.         0.         0.         ... 0.11111111 0.11111111 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.11111111 0.11111111 0.        ]
 ...
 [0.11111111 0.         0.11111111 ... 1.         0.22222222 0.        ]
 [0.11111111 0.         0.11111111 ... 0.22222222 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [221]:
# create list of indices for later matching
indices = pd.Series(df1.index, index = df1['title'])

In [222]:
user = pd.read_csv(u'userlist.csv')

df1.drop(['feature'],axis=1,inplace=True)
user = pd.merge(user, df1, on='movieid')
new = user.title.to_list()
user.head()

for i in new:
    print(i)

辛普森家庭電影版
海底總動員
魔法公主
回到未來


In [223]:
def recommend_movie(titles, n = 10, cosine_sim = cosine_sim):
    movies = []
    #for titles in userList
    for title in titles:
        # 檢索匹配的 movie_name index
        if title not in indices.index:
            print("Movie not in database.")
            return
        else:
            idx = indices[title]
        
        # 電影的餘弦相似度分數降序排列
        scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
            
        # 前 n 個最相似的 movies indexes
        # 使用 1:n 因為 0 是輸入的同一部電影
        top_n_idx = list(scores.iloc[1:n].index)
                
        #return result
        #print(df1['movie_name'].iloc[top_n_idx])
        ans = df1.iloc[top_n_idx]
        movies.append(ans)
    df = pd.concat(movies)    
    return df

In [224]:
ans = recommend_movie(new,7)
#ans
ans = ans.drop_duplicates(keep='first', inplace=False,subset=['title'])

In [225]:
ans.head()

,movieid,title,genres,string_agg
5854,255319,希達的秘密花園,"['家庭,動畫,喜劇']",['Sabine Azéma']
7120,364689,萌牛費迪南,"['冒險,喜劇,動畫,家庭']",['Anthony Anderson']
2387,14175,鴿戰總動員,"['戰爭,動畫,家庭,喜劇,冒險']",['Heidi Fecht']
2172,12233,奧麗華歷險記,"['喜劇,動畫,家庭']",['Joey Lawrence']
6295,295693,寶貝老闆,"['喜劇,動畫,家庭']",['Lisa Kudrow']


In [226]:
#清洗數據
from ast import literal_eval
ans['genres'] = ans['genres'].apply(literal_eval)
ans['string_agg'] = ans['string_agg'].apply(literal_eval)
ans['cast1'] = ''
ans['genres1'] = ''

user['genres'] = user['genres'].apply(literal_eval)
user['string_agg'] = user['string_agg'].apply(literal_eval)
user['cast1'] = ''
user['genres1'] = ''

def genres_words(x):
    return ('' + ' '.join(x['genres']))
ans['genres1'] = ans.apply(genres_words, axis = 1)
user['genres1'] = user.apply(genres_words, axis = 1)
def cast_words(x):
    return (' ' + ' '.join(x['string_agg']))
ans['cast1'] = ans.apply(cast_words, axis = 1)
user['genres1'] = user.apply(genres_words, axis = 1)

ans.drop(['genres','string_agg'],axis=1,inplace=True)
user.drop(['genres','string_agg'],axis=1,inplace=True)


In [227]:
ans.head()

,movieid,title,cast1,genres1
5854,255319,希達的秘密花園,Sabine Azéma,"家庭,動畫,喜劇"
7120,364689,萌牛費迪南,Anthony Anderson,"冒險,喜劇,動畫,家庭"
2387,14175,鴿戰總動員,Heidi Fecht,"戰爭,動畫,家庭,喜劇,冒險"
2172,12233,奧麗華歷險記,Joey Lawrence,"喜劇,動畫,家庭"
6295,295693,寶貝老闆,Lisa Kudrow,"喜劇,動畫,家庭"


In [228]:
#genres
featureG = ans.genres1.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)

/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [229]:
featureGU = user.genres1.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)

/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


In [239]:
#cast
featureC = ans.cast1.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)

/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [238]:
#cast
featureCU = user.cast1.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)

/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [240]:
movie_vec = featureG.merge(featureC, how='inner', left_index=True, right_index=True)
#movie_vec = movie_vec.merge(featureC, how='inner', left_index=True, right_index=True)
movie_vec = pd.concat([ans,movie_vec], axis=1)
movie_vec.set_index('movieid',inplace=True)
movie_vec.drop(['genres1','cast1','title'],axis=1,inplace=True)
movie_vec
#final = ans.drop_duplicates(keep='first', inplace=False,subset=['movie_name'])

,冒險,劇情,動作,動畫,喜劇,奇幻,家庭,戰爭,科幻,André Sogliuzzo,...,Keith Dallas,Kenta Miyake,Lisa Kudrow,Mateus Solano,Michael Patric,Miguel A. Núñez,Sabine Azéma,Sally Kinghorn,Tamao Nakamura,Walter Catlett
movieid,,,,,,,,,,,,,,,,,,,,,
255319,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
364689,1,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14175,1,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12233,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295693,0,0,0,1,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
462883,0,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
13378,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
436931,1,0,0,1,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
10800,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [254]:
user_vec = pd.DataFrame()
user_vec = user_vec.reindex(columns =movie_vec.columns)
user_vec = user_vec.merge(featureGU, how='outer')
#user_vec = user_vec.merge(featureDU, how='outer')
#user_vec = user_vec.merge(featureCU, how='outer')
#user_vec = user_vec.merge(featureGU, how='inner', left_index=True, right_index=True)
user_vec = user_vec.merge(featureCU, how='inner', left_index=True, right_index=True)
user_vec = pd.concat([user, user_vec], axis=1)
user_vec.drop(['genres1','cast1','movieid','title'],axis=1,inplace=True)
user_vec = user_vec.fillna(0.0)
user_vec

,userId,劇情,動作,戰爭,André Sogliuzzo,Anthony Anderson,Anushka Sharma,Bob Bergen,Caleb Hennigan,Ellen DeGeneres,...,Sabine Azéma,Sally Kinghorn,Tamao Nakamura,Walter Catlett,冒險,動畫,喜劇,奇幻,家庭,科幻
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1,1,0,1,0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1,0,0,1,0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1,0,1,0,0
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,1,1


In [255]:
user_vec = user_vec.groupby('userId').mean()
user_vec = user_vec[movie_vec.columns]
user_vec

,冒險,劇情,動作,動畫,喜劇,奇幻,家庭,戰爭,科幻,André Sogliuzzo,...,Keith Dallas,Kenta Miyake,Lisa Kudrow,Mateus Solano,Michael Patric,Miguel A. Núñez,Sabine Azéma,Sally Kinghorn,Tamao Nakamura,Walter Catlett
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.5,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.5,0.5,0.5,0.5,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [256]:
#user movie similar matrix
user_movie_matrix = cosine_similarity(user_vec.values,movie_vec.values)
user_movie_matrix = 1- user_movie_matrix 
user_movie_matrix = pd.DataFrame(user_movie_matrix, index=user_vec.index,columns=movie_vec.index)

In [257]:
import numpy as np
def get_the_most_similar_movies(user_id, user_movie_matrix,num):
 user_vec = user_movie_matrix.loc[user_id].values 
 sorted_index = np.argsort(user_vec)[:num]
 return list(user_movie_matrix.columns[sorted_index])

In [258]:
get_the_most_similar_movies(1, user_movie_matrix,4)

[255319, 12233, 295693, 462883]

In [259]:
user_movie_matrix

movieid,255319,364689,14175,12233,295693,462883,13378,436931,10800,10895,...,714194,528112,116711,13179,165,11260,381289,496327,682075,62177
userId,,,,,,,,,,,,,,,,,,,,,
1,0.166667,0.254644,0.319586,0.166667,0.166667,0.166667,0.2302,0.403715,0.2302,0.2302,...,0.701858,0.701858,0.403715,0.403715,0.552786,0.552786,0.591752,0.500000,0.6151,0.370059
2,0.500000,0.254644,0.319586,0.500000,0.500000,0.500000,0.6151,0.254644,0.6151,0.6151,...,0.403715,0.403715,0.254644,0.254644,0.254644,0.254644,0.319586,0.666667,0.6151,0.244071
